In [12]:
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
#extract the text data from the pdf for model making and testing and trials
def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob="*pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents

In [3]:
extracted_data = load_pdf_file(data='../Data/')

In [4]:
def text_split(extracted_data):
    text_spliter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks = text_spliter.split_documents(extracted_data)
    return text_chunks

In [5]:
text_chunks = text_split(extracted_data)
print(f"Length of the text chunks : ",len(text_chunks))

Length of the text chunks :  6295


In [33]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [34]:
def Hugging_face_embedding():
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [35]:
embeddings = Hugging_face_embedding()

/tmp/ipykernel_2014/141241356.py:2: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [52]:
import os
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv

load_dotenv()

PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

# ✅ New way to initialize client in v3.x
pc = Pinecone(api_key=PINECONE_API_KEY)

# ✅ Create index if it doesn't exist
index_name = "medbot"

if index_name not in [index.name for index in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

In [68]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [ ]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [53]:
from langchain_pinecone import PineconeVectorStore
dosearch = PineconeVectorStore.from_documents(
    documents= text_chunks,
    index_name = index_name,
    embedding=embeddings,
)

In [55]:
dosearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)
dosearch

In [57]:
retriever = dosearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [58]:
retrievr_docs = retriever.invoke("What is fever?")

In [59]:
print(retrievr_docs)

[Document(id='ee3d6d93-154c-482e-a15a-dacd44cccd54', metadata={'author': 'Clifford', 'creationdate': '2004-12-28T16:09:18-05:00', 'creator': 'PyPDF', 'enhanced': 'By PDF Enhancer 2.5/Win', 'moddate': '2006-07-11T00:27:43+02:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': '../Data/Alternative_Medicine_S_to_Z.pdf', 'spdf': '1096', 'total_pages': 666.0}, page_content='agitated, alternately hot and cold, thirsty , and patients\nwhose fever is worse after midnight.\n• Byronia 6c for the patient who is shivery and sweating,\nvery thirsty at long intervals, and having headaches and\npain.\n• F errum phosphoricum (iron phosphate) 6c for a mild\nfever of slow onset accompanied by frequent bouts of\nsweating, shivering, and headaches.\nW estern herbalism may be used to treat fever, but\ntreatment requires a qualified medical herbalist. The'), Document(id='cd3bd12b-e260-4671-b9a7-e9e6c5ec0c96', metadata={'author': 'Clifford', 'creationdate': '2004-12-28T16

In [79]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
GROQ_API_KEY = os.environ.get('GROQ_API_KEY')
os.environ["GROQ_API_KEY"] = GROQ_API_KEY  # or load from dotenv

llm = ChatGroq(
    model="llama3-8b-8192",  # or try "llama3-70b-8192" or "mixtral-8x7b-32768"
    temperature=0.4,
    max_tokens=500
)

In [81]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

systemPrompt =(
    "You are a helpful, smart, and safety-aware AI assistant."
    "You are an assistant for question answering task."
    "use the following piece of retrieved context answer"
    "the question If you don't know the answer , say that you"
    "don't know.Use two sentences maximum and keep the"
    "answer concise"
    "\n\n"
    "{question}"   
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",systemPrompt),
        ("human","{question}"),
    ]
)


In [82]:
chain = prompt | llm

In [83]:
response = chain.invoke({"question": "What are natural treatments for fever?"})
print(response.content)

Some natural treatments for fever include drinking plenty of fluids, such as water, clear broth, or electrolyte-rich beverages like coconut water or sports drinks, and using a lukewarm bath or shower to help bring down the body temperature. Additionally, over-the-counter pain relievers like acetaminophen or ibuprofen can be used, but it's essential to follow the recommended dosage and consult with a healthcare professional if necessary.
